**IF-B || Kecerdasan Buatan**<br>
Nama Kelompok: <br>
Jonathan Natannael Zefanya <br>
Daffa Danindra <br>
Ikhwan El-Faris <br>
Dhetalisa Aulia Kirana <br>

In [1]:
from google.colab import files

# Upload data training
print("Upload Data Training")
uploaded_train = files.upload()

# Upload data testing
print("Upload Data Testing")
uploaded_test = files.upload()

Upload Data Training


Saving Data_Training_AdultIncome.csv to Data_Training_AdultIncome.csv
Upload Data Testing


Saving Data_Testing_AdultIncome.csv to Data_Testing_AdultIncome.csv


In [2]:
# Fungsi untuk membuat tabel saat membaca data
from IPython.display import display, HTML

def tabel(df):
    display(HTML(df.to_html(index=False)))

In [3]:
# Membaca dan Menampilkan data training dan data testing
import pandas as pd

# Data Training
train_data = pd.read_csv(next(iter(uploaded_train.keys())))
print("Data Training:")
tabel(train_data)

# Data Testing
test_data = pd.read_csv(next(iter(uploaded_test.keys())))
print("Data Testing:")
tabel(test_data)

Data Training:


age,workclass,education,marital-status,occupation,relationship,gender,hours-per-week,native-country,income
30 - 39,Private,Some-college,Married-civ-spouse,Craft-repair,Husband,Male,40,United-States,<=50K
20 - 29,Private,11th,Married-civ-spouse,Transport-moving,Husband,Male,40,United-States,<=50K
40 - 49,Private,HS-grad,Married-civ-spouse,Adm-clerical,Husband,Male,40,United-States,<=50K
30 - 39,Private,Bachelors,Married-civ-spouse,Prof-specialty,Husband,Male,40,United-States,>50K
60 - 69,Private,Some-college,Married-civ-spouse,Handlers-cleaners,Husband,Male,15,United-States,<=50K
50 - 59,Private,HS-grad,Married-civ-spouse,Handlers-cleaners,Husband,Male,40,United-States,>50K
40 - 49,Local-gov,Doctorate,Never-married,Prof-specialty,Not-in-family,Female,40,United-States,>50K
30 - 39,Private,Some-college,Never-married,Craft-repair,Not-in-family,Male,40,United-States,<=50K
30 - 39,State-gov,HS-grad,Married-civ-spouse,Protective-serv,Other-relative,Female,40,United-States,<=50K
50 - 59,Private,10th,Divorced,Other-service,Not-in-family,Female,20,United-States,<=50K


Data Testing:


age,workclass,education,marital-status,occupation,relationship,gender,hours-per-week,native-country,income
20 - 29,Private,Some-college,Never-married,Other-service,Own-child,Male,15,United-States,<=50K
20 - 29,Local-gov,Some-college,Never-married,Protective-serv,Own-child,Female,40,United-States,<=50K
50 - 59,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,Male,40,United-States,<=50K
30 - 39,Private,HS-grad,Divorced,Adm-clerical,Unmarried,Female,40,United-States,<=50K
40 - 49,Private,Bachelors,Married-civ-spouse,Prof-specialty,Husband,Male,40,United-States,>50K
30 - 39,Local-gov,Some-college,Married-civ-spouse,Craft-repair,Husband,Male,40,United-States,<=50K
20 - 29,Private,Bachelors,Never-married,Prof-specialty,Own-child,Female,72,United-States,<=50K
20 - 29,Private,HS-grad,Never-married,Sales,Not-in-family,Female,35,United-States,<=50K
40 - 49,Private,Some-college,Widowed,Adm-clerical,Not-in-family,Female,60,United-States,<=50K
40 - 49,Private,HS-grad,Married-civ-spouse,Transport-moving,Husband,Male,45,United-States,<=50K


In [4]:
# Menghitung probabilitas dari setiap target
# P(Ci)
probabilitas = train_data['income'].value_counts(normalize=True)
print(f"Probabilitas X:\n{probabilitas}",)

Probabilitas X:
income
<=50K    0.764706
>50K     0.235294
Name: proportion, dtype: float64


In [5]:
# Dictionary untuk menyimpan value/hasil dari P(X|Ci)
probabilitas_kondisional = {}
atributs = train_data.columns[:-1]  # Semua kolom kecuali kolom 'income'
laplace = 1  # Nilai 1 adalah α (alpha) , yaitu nilai kecil yang ditambahkan agar tidak ada zero probability (probabilitas dengan nilai 0).

In [6]:
# Menghitung semua atribut/kondisi P(X)
for atribut in atributs:
    probabilitas_kondisional[atribut] = {}
    for value in train_data[atribut].unique():
        # Menghitung probabilitas kondisional untuk setiap nilai pada atribut
        probabilitas_kondisional[atribut][value] = {}
        for target in probabilitas.index:
            # Inisialisasi target "income"
            subset = train_data[train_data['income'] == target]
            # Jika ada atribut yang memiliki nilai 0 maka menggunakan laplaceian
            count_value = subset[atribut].value_counts().get(value, 0) + laplace
            # Nilai X didapat dari jumlah atribut yang bernilai 0 (laplace * len(train_data[atribut].unique()))
            total_count = subset.shape[0] + (laplace * len(train_data[atribut].unique()))
            # Menghitung probabilitas kondisional (P(X|Ci)) dari setiap atribut
            probabilitas_kondisional[atribut][value][target] = count_value / total_count

In [7]:
print("Probabilitas Kondisional:\n", probabilitas_kondisional)

Probabilitas Kondisional:
 {'age': {'30 - 39': {'<=50K': 0.2222222222222222, '>50K': 0.16666666666666666}, '20 - 29': {'<=50K': 0.3333333333333333, '>50K': 0.2222222222222222}, '40 - 49': {'<=50K': 0.2, '>50K': 0.2777777777777778}, '60 - 69': {'<=50K': 0.1111111111111111, '>50K': 0.1111111111111111}, '50 - 59': {'<=50K': 0.08888888888888889, '>50K': 0.16666666666666666}, '70 - 79': {'<=50K': 0.044444444444444446, '>50K': 0.05555555555555555}}, 'workclass': {'Private': {'<=50K': 0.6363636363636364, '>50K': 0.47058823529411764}, 'Local-gov': {'<=50K': 0.11363636363636363, '>50K': 0.23529411764705882}, 'State-gov': {'<=50K': 0.13636363636363635, '>50K': 0.11764705882352941}, 'Self-emp-not-inc': {'<=50K': 0.06818181818181818, '>50K': 0.11764705882352941}, '?': {'<=50K': 0.045454545454545456, '>50K': 0.058823529411764705}}, 'education': {'Some-college': {'<=50K': 0.23529411764705882, '>50K': 0.08333333333333333}, '11th': {'<=50K': 0.0392156862745098, '>50K': 0.041666666666666664}, 'HS-grad'

In [8]:
# Fungsi klasifikasi menggunakan Naive Bayes dengan Laplace Smoothing
def naive_bayes(instance):
    target = probabilitas.index
    probabilities = {}

    for tgt in target:
        prob = probabilitas[tgt] # P(Ci)
        for atribut in atributs:
            if instance[atribut] in probabilitas_kondisional[atribut]: # P(X|Ci)
                # P(Ci|X) = P(Ci) * P(X|Ci)
                prob *= probabilitas_kondisional[atribut][instance[atribut]].get(tgt, 1)  # Tambahkan smoothing dengan nilai sangat kecil
            else:
                prob *= 1  # Tambahkan smoothing dengan nilai sangat kecil
        probabilities[tgt] = prob

    predicted_target = max(probabilities, key=probabilities.get)
    return predicted_target, probabilities

In [9]:
# Klasifikasi data pengujian
results = test_data.apply(naive_bayes, axis=1)

In [10]:
# Menyimpan hasil ke dalam dataframe
test_data['predicted_target'] = results.apply(lambda x: x[0])
test_data['probabilities'] = results.apply(lambda x: x[1])

In [11]:
# Cetak semua informasi probabilitas dari semua jenis target
for index, row in test_data.iterrows():
    print(f"Data ke-{index + 1} dengan atribut {row[atributs].to_dict()}")
    for tgt, prob in row['probabilities'].items():
        print(f"  Probabilitas dari target {tgt}: {prob}")
    print(f"  Prediksi Target: {row['predicted_target']}\n")

Data ke-1 dengan atribut {'age': '20 - 29', 'workclass': 'Private', 'education': 'Some-college', 'marital-status': 'Never-married', 'occupation': 'Other-service', 'relationship': 'Own-child', 'gender': 'Male', 'hours-per-week': 15, 'native-country': 'United-States'}
  Probabilitas dari target <=50K: 1.0630585912919964e-05
  Probabilitas dari target >50K: 1.2846077744485664e-08
  Prediksi Target: <=50K

Data ke-2 dengan atribut {'age': '20 - 29', 'workclass': 'Local-gov', 'education': 'Some-college', 'marital-status': 'Never-married', 'occupation': 'Protective-serv', 'relationship': 'Own-child', 'gender': 'Female', 'hours-per-week': 40, 'native-country': 'United-States'}
  Probabilitas dari target <=50K: 1.9713311789068617e-06
  Probabilitas dari target >50K: 8.56405182965711e-09
  Prediksi Target: <=50K

Data ke-3 dengan atribut {'age': '50 - 59', 'workclass': 'Private', 'education': 'HS-grad', 'marital-status': 'Married-civ-spouse', 'occupation': 'Machine-op-inspct', 'relationship': '

In [12]:
# Menampilkan hasil
print("Hasil Klasifikasi:")
tabel(test_data[["age","workclass","education","marital-status","occupation","relationship","gender","hours-per-week","native-country", 'predicted_target', 'probabilities']])

Hasil Klasifikasi:


age,workclass,education,marital-status,occupation,relationship,gender,hours-per-week,native-country,predicted_target,probabilities
20 - 29,Private,Some-college,Never-married,Other-service,Own-child,Male,15,United-States,<=50K,"{'<=50K': 1.0630585912919964e-05, '>50K': 1.2846077744485664e-08}"
20 - 29,Local-gov,Some-college,Never-married,Protective-serv,Own-child,Female,40,United-States,<=50K,"{'<=50K': 1.9713311789068617e-06, '>50K': 8.56405182965711e-09}"
50 - 59,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,Male,40,United-States,<=50K,"{'<=50K': 1.4415908269363623e-05, '>50K': 1.1099011171235617e-05}"
30 - 39,Private,HS-grad,Divorced,Adm-clerical,Unmarried,Female,40,United-States,<=50K,"{'<=50K': 9.379928746694083e-07, '>50K': 1.2846077744485669e-08}"
40 - 49,Private,Bachelors,Married-civ-spouse,Prof-specialty,Husband,Male,40,United-States,>50K,"{'<=50K': 1.9960488372965016e-05, '>50K': 0.00011099011171235617}"
30 - 39,Local-gov,Some-college,Married-civ-spouse,Craft-repair,Husband,Male,40,United-States,>50K,"{'<=50K': 7.920828719430563e-06, '>50K': 1.3873763964044521e-05}"
20 - 29,Private,Bachelors,Never-married,Prof-specialty,Own-child,Female,72,United-States,<=50K,"{'<=50K': 2.371438395959069e-05, '>50K': 7.964568201581113e-07}"
20 - 29,Private,HS-grad,Never-married,Sales,Not-in-family,Female,35,United-States,<=50K,"{'<=50K': 3.764999177492487e-06, '>50K': 1.712810365931422e-08}"
40 - 49,Private,Some-college,Widowed,Adm-clerical,Not-in-family,Female,60,United-States,<=50K,"{'<=50K': 5.1517454808765994e-06, '>50K': 4.014399295151771e-08}"
40 - 49,Private,HS-grad,Married-civ-spouse,Transport-moving,Husband,Male,45,United-States,>50K,"{'<=50K': 5.405965601011359e-06, '>50K': 6.9368819820222605e-06}"
